In [2]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import dask
import trend_utils as tu
from regridding import regrid
from utils import loca_gard_mapping, trend_metric_ids as metric_ids

In [9]:
################
#### Paths #####
################
# Update these for reproduction

project_data_path = "/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024"
project_code_path = "/storage/home/dcl5300/work/current_projects/conus_comparison_lafferty-etal-2024"

In [10]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="10GiB",
    walltime="00:10:00",
)

cluster.scale(jobs=1)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.170:37853,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [11]:
cluster.scale(jobs=5)

## Fit trend

In [30]:
# Fit all
for metric_id in metric_ids:
    tu.trend_fit_all(metric_id)

## Regridding

In [31]:
%%time
# Regrid to LOCA using NN
method = "nearest"
target = "LOCA2"

for metric_id in metric_ids:
    # Save path
    store_path = f"{project_data_path}/trends/loca_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/trends/original_grids/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/trends/original_grids/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            try:
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid(ds_tmp, method=method, target=target)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
            except Exception as e:
                print(file)

CPU times: user 74.9 ms, sys: 41.6 ms, total: 116 ms
Wall time: 441 ms


In [32]:
%%time
# Regrid to GARD-LENS using NN
method = "nearest"
target = "GARD-LENS"

for metric_id in metric_ids:
    # Save path
    store_path = f"{project_data_path}/trends/gard_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/trends/original_grids/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/trends/original_grids/{metric_id}/LOCA2*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            try:
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid(ds_tmp, method=method, target=target)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
            except Exception as e:
                print(file)

CPU times: user 48 s, sys: 9.73 s, total: 57.7 s
Wall time: 1min 43s
